# PROYECTO PRODUCCIÓN AGRÍCOLA SEGÚN LA METEOROLOGÍA

### 1. Unificación y Limpieza de Datos

En esta sección, unificaremos y limpiaremos los datasets meteorológicos y agrícolas. Posteriormente, guardaremos el dataset unificado para su uso en las siguientes fases del proyecto.


In [4]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [5]:
# Cargar los datasets
agri_df = pd.read_csv('/workspaces/proyectfinaltour/data/processed/datasets/agricola_procesado.csv')
weather_df = pd.read_csv('/workspaces/proyectfinaltour/data/processed/datasets/weather.csv')

# Verificar las columnas presentes en cada DataFrame
print("Columnas en el dataset agrícola:", agri_df.columns)
print("Columnas en el dataset meteorológico:", weather_df.columns)

# Crear las columnas 'year' y 'month' en el dataset meteorológico
weather_df['date_time'] = pd.to_datetime(weather_df['date_time'])
weather_df['year'] = weather_df['date_time'].dt.year
weather_df['month'] = weather_df['date_time'].dt.month

# Verificar si las columnas se crearon correctamente
print("Columnas en el dataset meteorológico después de añadir 'year' y 'month':", weather_df.columns)

# Unificar los datasets
combined_df = pd.merge(agri_df, weather_df, left_on=['Ciudad', 'year', 'Mes'], right_on=['city', 'year', 'month'], how='inner')

# Guardar el dataset unificado
combined_df.to_csv('/workspaces/proyectfinaltour/data/processed/datasets/combined_dataset.csv', index=False)

print("Dataset unificado guardado correctamente.")


Columnas en el dataset agrícola: Index(['Ciudad', 'Cultivo', 'year', 'Mes', 'Kilos', 'Hectáreas', 'date'], dtype='object')
Columnas en el dataset meteorológico: Index(['city', 'date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset',
       'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC',
       'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM',
       'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKmph'],
      dtype='object')
Columnas en el dataset meteorológico después de añadir 'year' y 'month': Index(['city', 'date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset',
       'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC',
       'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM',
       'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKm

In [6]:
# Cargar el dataset unificado
combined_df = pd.read_csv('/workspaces/proyectfinaltour/data/processed/datasets/combined_dataset.csv')

# Mostrar las primeras filas 
print(combined_df.head())

   Ciudad Cultivo  year  Mes  Kilos   Hectáreas        date    city  \
0  Bilbao   Olivo  2008    7   1769  363.779398  2008-07-15  Bilbao   
1  Bilbao   Olivo  2008    7   1769  363.779398  2008-07-15  Bilbao   
2  Bilbao   Olivo  2008    7   1769  363.779398  2008-07-15  Bilbao   
3  Bilbao   Olivo  2008    7   1769  363.779398  2008-07-15  Bilbao   
4  Bilbao   Olivo  2008    7   1769  363.779398  2008-07-15  Bilbao   

    date_time  maxtempC  ...  WindGustKmph  cloudcover  humidity  precipMM  \
0  2008-07-01        22  ...            14          18        82       0.2   
1  2008-07-02        18  ...            13          71        88       2.2   
2  2008-07-03        18  ...            22          30        74       0.5   
3  2008-07-04        26  ...            22          19        63       0.0   
4  2008-07-05        18  ...            20          80        89       1.4   

   pressure  tempC visibility winddirDegree windspeedKmph month  
0      1015     22         10         

In [7]:
# Eliminar la columna 'date' si ya no es necesaria
if 'date' in combined_df.columns:
    combined_df = combined_df.drop(columns=['date'])

# Verificar las columnas después de eliminar 'date'
print(combined_df.columns)

Index(['Ciudad', 'Cultivo', 'year', 'Mes', 'Kilos', 'Hectáreas', 'city',
       'date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour',
       'uvIndex', 'uvIndex.1', 'moon_illumination', 'moonrise', 'moonset',
       'sunrise', 'sunset', 'DewPointC', 'FeelsLikeC', 'HeatIndexC',
       'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity', 'precipMM',
       'pressure', 'tempC', 'visibility', 'winddirDegree', 'windspeedKmph',
       'month'],
      dtype='object')


In [8]:
# Agrupar por Ciudad, Cultivo, year y Mes
grouped_df = combined_df.groupby(['Ciudad', 'Cultivo', 'year', 'Mes']).agg({
    'Kilos': 'sum',
    'Hectáreas': 'sum',
    'tempC': 'mean',
    'precipMM': 'mean',
    'sunHour': 'mean',
    'uvIndex': 'mean'
}).reset_index()

# Calcular la producción por hectárea
grouped_df['produccion_por_hectarea'] = grouped_df['Kilos'] / grouped_df['Hectáreas']

# Mostrar las primeras filas del dataset agrupado
print(grouped_df.head())


      Ciudad Cultivo  year  Mes    Kilos     Hectáreas      tempC  precipMM  \
0  Barcelona    Maíz  2008    7   973493  15421.892611  25.451613  0.300000   
1  Barcelona    Maíz  2008    8   227664  15257.159949  26.677419  0.312903   
2  Barcelona    Maíz  2008    9   646140  14428.691635  23.166667  0.463333   
3  Barcelona    Maíz  2008   10   205189    779.942823  19.806452  1.696774   
4  Barcelona    Maíz  2008   11  1127250   5303.690433  14.200000  1.006667   

     sunHour   uvIndex  produccion_por_hectarea  
0  14.435484  5.935484                63.124094  
1  12.270968  6.096774                14.921781  
2  10.993333  5.433333                44.781607  
3  10.180645  4.516129               263.082105  
4   9.630000  3.800000               212.540685  


In [9]:
# Guardar el dataset agrupado
grouped_df.to_csv('/workspaces/proyectfinaltour/data/processed/datasets/combined_dataset.csv', index=False)

print("Dataset agrupado y guardado correctamente.")


Dataset agrupado y guardado correctamente.


In [10]:
# Análisis descriptivo de las variables
print(combined_df.describe())

                year            Mes          Kilos      Hectáreas  \
count  107408.000000  107408.000000  107408.000000  107408.000000   
mean     2013.236444       6.666058   25568.094956     253.208748   
std         3.038718       3.444694   14116.553935     141.020283   
min      2008.000000       1.000000    1002.000000      10.066000   
25%      2011.000000       4.000000   13449.000000     130.445645   
50%      2013.000000       7.000000   25476.000000     255.176129   
75%      2016.000000      10.000000   38008.000000     372.906813   
max      2018.000000      12.000000   49998.000000     499.859764   

            maxtempC       mintempC   totalSnow_cm        sunHour  \
count  107408.000000  107408.000000  107408.000000  107408.000000   
mean       20.508714      12.450283       0.002361      10.634653   
std         7.333967       5.504573       0.065104       2.846374   
min         0.000000      -6.000000       0.000000       3.400000   
25%        15.000000       8.0000

In [11]:
# Gráfico de la evolución de la temperatura a lo largo de los años para diferentes ciudades
plt.figure(figsize=(10, 6))
sns.lineplot(data=combined_df, x='date_time', y='tempC', hue='Ciudad')
plt.title('Evolución de la Temperatura por Ciudad')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (C°)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Gráfico de la evolución de la producción agrícola a lo largo de los años para diferentes cultivos
plt.figure(figsize=(10, 6))
sns.lineplot(data=combined_df, x='date_time', y='Kilos', hue='Cultivo')
plt.title('Evolución de la Producción Agrícola por Cultivo')
plt.xlabel('Fecha')
plt.ylabel('Producción (Kilos)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Gráfico de la evolución de la producción agrícola a lo largo de los años para diferentes cultivos
plt.figure(figsize=(10, 6))
sns.lineplot(data=combined_df, x='date_time', y='Kilos', hue='Cultivo')
plt.title('Evolución de la Producción Agrícola por Cultivo')
plt.xlabel('Fecha')
plt.ylabel('Producción (Kilos)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Gráfico de la evolución de la producción agrícola a lo largo de los años para diferentes cultivos
plt.figure(figsize=(10, 6))
sns.lineplot(data=combined_df, x='date_time', y='Kilos', hue='Cultivo')
plt.title('Evolución de la Producción Agrícola por Cultivo')
plt.xlabel('Fecha')
plt.ylabel('Producción (Kilos)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Gráfico de la evolución de las horas de sol a lo largo de los años para diferentes ciudades
plt.figure(figsize=(10, 6))
sns.lineplot(data=combined_df, x='date_time', y='sunHour', hue='Ciudad')
plt.title('Evolución de las Horas de Sol por Ciudad')
plt.xlabel('Fecha')
plt.ylabel('Horas de Sol')
plt.xticks(rotation=45)
plt.show()